In [2]:
!python -m spacy download es_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.9 MB 4.7 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [3]:
import pandas as pd
import string as st
import spacy
import time
import re
import nltk
from nltk.stem import SnowballStemmer
from nltk import PorterStemmer, WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
import es_core_news_sm
sp = es_core_news_sm.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Let's us read the input data
df = pd.read_csv('/content/drive/MyDrive/TFM/data/export_expedientes.csv', sep=',', encoding='UTF8')
df.head()

,id_capitulo,id_concepto,id_partida_generica,descripcion
0,3000,3300,3390,SERVICIOS PROFESIONALES CONSISTENTES EN LA COO...
1,3000,3300,3390,SERVICIOS PROFESIONALES CONSISTENTES EN COADYU...
2,3000,3300,3390,SERVICIOS PROFESIONALES CONSISTENTES EN COADYU...
3,3000,3300,3390,SERVICIOS PROFESIONALES CONSISTENTES EN COADYU...
4,3000,3300,3390,SERVICIOS PROFESIONALES CONSISTENTES EN COADYU...


In [ ]:
df.isnull().sum()

id_capitulo            0
id_concepto            0
id_partida_generica    0
descripcion            0
dtype: int64

In [ ]:
#Solo para probar el tiempo que tardan los metodos del preprocesamiento
df = df.sample(n=50000, random_state=1)
df

,id_capitulo,id_concepto,id_partida_generica,descripcion
246729,2000,2500,2530,Infliximab SOLUCIÓN INYECTABLE El frasco ámpul...
71686,3000,3200,3250,SERVICIO DE ARRENDAMIENTO INTEGRAL DE VEHÍCULO...
26815,6000,6100,6150,Invitación E350-2018 Seguimiento y control par...
294323,3000,3500,3510,TRABAJOS DE REHABILTACIÓN Y MANTTO. EN LA ESTA...
224244,2000,2500,2530,ADQUISICION DE INSUMOS DE LISOSOMALES (IDURSUL...
...,...,...,...,...
496893,3000,3300,3390,PRESTACIÓN DE SERVICIOS PROFESIONALES DGAQ
299737,3000,3200,3270,Suscripción anual del servicio de correo elect...
73851,3000,3500,3580,CONTRATACIÓN PLURIANUAL DEL SERVICIO INTEGRAL ...
157176,3000,3100,3170,CONTRATACION DE SERVICIO DE CONDUCCION DE SEÑA...


In [ ]:
df.groupby(['id_capitulo']).count()['descripcion']

id_capitulo
2000    271604
3000    269757
5000     17605
6000     67202
Name: descripcion, dtype: int64

# Text cleaning and processing steps
* Remove punctuations
* Convert text to tokens
* Remove tokens of length less than or equal to 3
* Remove stopwords using NLTK corpus stopwords list to match
* Apply stemming
* Apply lemmatization
* Convert words to feature vectors

In [ ]:
# Remove all punctuations from the text
def remove_punct(text):
    return ("".join([ch for ch in text if ch not in st.punctuation]))

''' Convert text to lower case tokens. Here, split() is applied on white-spaces. But, it could be applied
    on special characters, tabs or any other string based on which text is to be seperated into tokens.
'''
def tokenize(text):
    text = re.split('\s+' ,text)
    return [x.lower() for x in text]

# Remove tokens of length less than 3
def remove_small_words(text):
    return [x for x in text if len(x) > 3 ]

''' Remove stopwords. Here, NLTK corpus list is used for a match. However, a customized user-defined 
    list could be created and used to limit the matches in input text. 
'''
def remove_stopwords(text):
  return [word for word in text if word not in nltk.corpus.stopwords.words('spanish')]

# Apply stemming to get root words 
def stemming(text):
    stemmer = SnowballStemmer('spanish')
    return [stemmer.stem(word) for word in text]
  
# Apply lemmatization on tokens
def lemmatize_en(text):
    word_net = WordNetLemmatizer()
    return [word_net.lemmatize(word) for word in text]

def lemmatize_es(text):
    sp = es_core_news_sm.load()
    return [sp(word)[0].lemma_ for word in text]
  
# Annotate each word with its part-of-speech tag
def get_pos_tag_en(tokenized_sentence):
  return nltk.pos_tag(tokenized_sentence)

# Annotate each word with its part-of-speech tag
def get_pos_tag_es(text):
  sp = es_core_news_sm.load()
  return [sp(word)[0].pos_ for word in text]

def lemmatize_es(text):
  sp = es_core_news_sm.load()
  return [sp(word)[0].lemma_ for word in text]

# Create sentences to get clean text as input for vectors
def return_sentences(tokens):
    return " ".join([word for word in tokens])

In [ ]:
df['des_removed_punc'] = df['descripcion'].apply(lambda x: remove_punct(x))
df['tokens1'] = df['des_removed_punc'].apply(lambda msg : tokenize(msg))
#An alternate method to tokenizing that resorts to resources provided by NLTK
df['tokens'] = df['des_removed_punc'].apply(lambda msg : word_tokenize(msg.lower(), "spanish"))
df['larger_tokens'] = df['tokens'].apply(lambda x : remove_small_words(x))
df['clean_tokens'] = df['larger_tokens'].apply(lambda x : remove_stopwords(x))
### Apply stemming to convert tokens to their root form. This is a rule-based process 
###of word form conversion where word-suffixes are truncated irrespective of whether the root word is an actual word in the language dictionary. 
##### Note that this step is optional and depends on problem type.
df['stem_words'] = df['clean_tokens'].apply(lambda wrd: stemming(wrd))
###Lemmatization converts word to it's dictionary base form. This process takes language grammar and vocabulary into consideration while conversion. 
###Hence, it is different from Stemming in that it does not merely truncate the suffixes to get the root word.
df['lemma_words'] = df['clean_tokens'].apply(lambda x : lemmatize_es(x))
#Let us now annotate each token in a document with its Part-Of-Speech tag (note that tokenized FULL sentences are required!)
df['pos_tag'] = df['clean_tokens'].apply(lambda x : get_pos_tag_es(x))
df['clean_text'] = df['lemma_words'].apply(lambda x : return_sentences(x))

In [ ]:
def preprocessing(text):
  des_removed_punc = remove_punct(text.lower())
  tokens = nltk.word_tokenize(des_removed_punc, "spanish")
  larger_tokens = remove_small_words(tokens)
  clean_tokens = remove_stopwords(larger_tokens)
  lemma_words = lemmatize_es(clean_tokens)
  clean_text = return_sentences(lemma_words)
  return clean_text

In [ ]:
def preprocessing_faster(text):
  #tokenizamos words 
  token_word = nltk.word_tokenize(text.lower(), "spanish")
  index = 0
  while(index < len(token_word)):
    #omit tokens if are less than 4 characters or are punctuation marks
    if len(token_word[index]) <= 3 or token_word[index] in st.punctuation:
      token_word.pop(index)
    elif not (token_word[index].isalpha()):
      token_word.pop(index)
    #omit tokens if are stopwords in spanish
    elif token_word[index] in nltk.corpus.stopwords.words('spanish'):
      token_word.pop(index)
    else:
      #lematization of tokens
      lemma_word = sp(token_word[index])[0].lemma_
      token_word[index] = lemma_word
      index += 1
  return ' '.join(token_word)

In [ ]:
def preprocessing_fastest(text):
  #tokenizamos words 
  token_word = nltk.word_tokenize(text.lower(), "spanish")
  index = 0
  while(index < len(token_word)):
    #omit tokens if are less than 4 characters or are punctuation marks
    if len(token_word[index]) <= 3 or token_word[index] in st.punctuation:
      token_word.pop(index)
    elif not (token_word[index].isalpha()):
      token_word.pop(index)
    #omit tokens if are stopwords in spanish
    elif token_word[index] in nltk.corpus.stopwords.words('spanish'):
      token_word.pop(index)
    else:
      index += 1
  #lematization of tokens
  doc = sp(' '.join(token_word))
  #for token in doc:
  #  print(token.text, token.lemma_, token.pos_, token.dep_)
  return " ".join([token.lemma_ for token in doc])

In [ ]:
def preprocessing_fastest_v2(text):
  #tokenizamos words 
  token_word = nltk.word_tokenize(text.lower(), "spanish")
  sentence = []
  for token in token_word:
    #omit tokens if are less than 4 characters or are punctuation marks
    if len(token) <= 3 or token in st.punctuation:
      continue
    #omit if the token has numbers
    if not (token.isalpha()):
      continue
    #omit tokens if are stopwords in spanish
    if token in nltk.corpus.stopwords.words('spanish'):
      continue
    sentence.append(token)
  #lematization of sentence
  doc = sp(' '.join(sentence))
  return ' '.join([token.lemma_ for token in doc])

In [ ]:
print(df['descripcion'].values[0])

SERVICIOS PROFESIONALES CONSISTENTES EN LA COORDINACIÓN DEL PROGRAMA CULTURAL DEL PROYECTO BOSQUE DE CHAPULTEPEC NATURALEZA Y CULTURA, A TRAVÉS DE SUS EJES: LA CONEXIÓN ENTRE LO BIOLÓGICO Y LO CULTURAL, UN PLAN INTEGRAL DE MOVILIDAD ENTRE LAS CUATRO SECCIONES Y SU ENTORNO URBANO, LA PROYECCIÓN DE UN ESPACIO DE POLÍTICA AMBIENTAL Y ESPACIO PÚBLICO CULTURAL, Y UN ESPACIO PÚBLICO CON DIVERSA OFERTA CULTURAL, HISTÓRICA, AMBIENTAL Y RECREATIVA PARA FAVORECER EL DESARROLLO DE ACTIVIDADES QUE MEJOREN LA CALIDAD DE VIDA DE LA POBLACIÓN DE TODO EL PAÍS E IMPULSE LA CONVIVENCIA E INCLUSIÓN SOCIAL Y DIVERSA. 


In [ ]:
ejemploLower = df['descripcion'].values[0].lower()
ejemploLower = remove_punct(ejemploLower)
tokens = nltk.word_tokenize(ejemploLower, "spanish")
larger_tokens = remove_small_words(tokens)
clean_tokens = remove_stopwords(larger_tokens)
lemma_words = lemmatize_es(clean_tokens)
clean_text = return_sentences(lemma_words)
print(clean_text)

servicio profesional consistent coordinación programa cultural proyecto bosque chapultepec naturaleza cultura través eje conexión biológico cultural plan integral movilidad cuatro sección entorno urbano proyección espacio político ambiental espacio público cultural espacio público diverso oferta cultural histórico ambiental recreativo favorecer desarrollo actividad mejorar calidad vida población país impulse convivencia inclusión social diverso


In [ ]:
print(preprocessing(df['descripcion'].values[0]))

servicio profesional consistent coordinación programa cultural proyecto bosque chapultepec naturaleza cultura través eje conexión biológico cultural plan integral movilidad cuatro sección entorno urbano proyección espacio político ambiental espacio público cultural espacio público diverso oferta cultural histórico ambiental recreativo favorecer desarrollo actividad mejorar calidad vida población país impulse convivencia inclusión social diverso


In [ ]:
print(preprocessing_fastest_v2(df['descripcion'].values[0]))

servicio profesional consistente coordinación programa cultural proyecto bosque chapultepec naturaleza culturar través eje conexión biológico cultural plan integral movilidad cuatro sección entorno urbano proyección espacio político ambiental espacio público cultural espacio público diverso oferta cultural histórico ambiental recreativo favorecer desarrollo actividad mejorar calidad vido población país impulse convivencia inclusión social diverso


In [ ]:
print(df['descripcion'].values[0])

ACTIVIDADES ESPECIALIZADAS PARA LA PRESTACIÓN DE SERVICIOS DE DEFENSORIA DE AUDIENCIAS DE RADIO EDUCACIÓN, PARA SUS SEÑALES HERTZIANANS: 1060 AM, EN LA CDMX, ONDA CORTA 6185 KHZ BI 49 M; 107.9 FM, EN MERIDA, YUCATÁN Y 96.5 FM EN LA CDMX-DE PRÓXIMA INSTALACIÓN. ASÍ COMO A TRAVÉS, DE SU SEÑAL SATELITAL -CANAL 2 DE LA RED EDUSAT- Y SUS SEÑALES DIGITALES DE RADIO EN LÍNEA COMO SE INDICA EN EL PUNTO V DE SU DICTAMEN DE JUSTIFICACIÓN.


In [ ]:
#Version origianl, es una version lenta
start = time.time()
print(preprocessing(df['descripcion'].values[0]))
end = time.time()
print(end - start)


actividad especializar prestación servicio defensoria audiencia radiar educación señalar hertzianans 1060 cdmx onda corto 6185 1079 merida yucatán cdmxde próximo instalación través señal satelital canal edusat señalar digital radiar líneo indicar punto dictamen justificación
2.234009027481079


In [ ]:
#Primera versión mejorada, más rápida
start = time.time()
print(preprocessing_faster(df['descripcion'].values[0]))
end = time.time()
print(end - start)

actividad especializar prestación servicio defensoria audiencia radiar educación señalar hertzianans cdmx onda corto merida yucatán próximo instalación través señal satelital señalar digital radiar líneo indicar punto dictamen justificación
0.21491718292236328


In [ ]:
#Segunada version super mejorada: Más rápida
start = time.time()
print(preprocessing_fastest(df['descripcion'].values[0]))
end = time.time()
print(end - start)

actividad especializar prestación servicio defensoria audiencia radiar educación señalar hertzianans cdmx onda corto merida yucatán próximo instalación través señal satelital señalar digital radiar líneo indicar punto dictamen justificación
0.031661272048950195


In [ ]:
#Tercera version super mejorada: La más rápida
start = time.time()
print(preprocessing_fastest_v2(df['descripcion'].values[0]))
end = time.time()
print(end - start)

actividad especializar prestación servicio defensoria audiencia radiar educación señalar hertzianans cdmx onda corto merida yucatán próximo instalación través señal satelital señalar digital radiar líneo indicar punto dictamen justificación
0.02961874008178711


In [ ]:
#Versión original con 1000 muestras
start = time.time()
df['clean_text_v1'] = df['descripcion'].apply(lambda description: preprocessing(description))
end = time.time()
print(end - start)

1973.0709493160248


In [ ]:
#Versión rápida con 1000 muestras
start = time.time()
df['clean_text_v2'] = df['descripcion'].apply(lambda description: preprocessing_faster(description))
end = time.time()
print(end - start)

81.75307512283325


In [ ]:
#Versión más rápida
start = time.time()
df['clean_text_V3'] = df['descripcion'].apply(lambda description: preprocessing_fastest(description))
end = time.time()
print(end - start)

10.928874492645264


In [ ]:
#Versión la más rápida
start = time.time()
df['clean_text'] = df['descripcion'].apply(lambda description: preprocessing_fastest_v2(description))
end = time.time()
print(end - start)

543.0309081077576


In [ ]:
df['clean_text'] = df['descripcion'].apply(lambda description: preprocessing_fastest_v2(description))

In [ ]:
df

,id_capitulo,id_concepto,id_partida_generica,descripcion,clean_text
246729,2000,2500,2530,Infliximab SOLUCIÓN INYECTABLE El frasco ámpul...,infliximab solución inyectable frasco ámpula l...
71686,3000,3200,3250,SERVICIO DE ARRENDAMIENTO INTEGRAL DE VEHÍCULO...,servicio arrendamiento integral vehículo terre...
26815,6000,6100,6150,Invitación E350-2018 Seguimiento y control par...,invitación seguimiento control trabajo relativ...
294323,3000,3500,3510,TRABAJOS DE REHABILTACIÓN Y MANTTO. EN LA ESTA...,trabajo rehabiltación mantto estancia bienesta...
224244,2000,2500,2530,ADQUISICION DE INSUMOS DE LISOSOMALES (IDURSUL...,adquisicion insumo lisosomales idursulfasa elo...
...,...,...,...,...,...
496893,3000,3300,3390,PRESTACIÓN DE SERVICIOS PROFESIONALES DGAQ,prestación servicio profesional dgaq
299737,3000,3200,3270,Suscripción anual del servicio de correo elect...,suscripción anual servicio correar electrónico...
73851,3000,3500,3580,CONTRATACIÓN PLURIANUAL DEL SERVICIO INTEGRAL ...,contratación plurianual servicio integral mane...
157176,3000,3100,3170,CONTRATACION DE SERVICIO DE CONDUCCION DE SEÑA...,contratacion servicio conduccion señalar analo...


In [4]:
df.head()

,id_capitulo,id_concepto,id_partida_generica,descripcion,clean_text
0,3000,3300,3390,SERVICIOS PROFESIONALES CONSISTENTES EN LA COO...,servicio profesional consistente coordinación ...
1,3000,3300,3390,SERVICIOS PROFESIONALES CONSISTENTES EN COADYU...,servicio profesional consistente coadyuvar des...
2,3000,3300,3390,SERVICIOS PROFESIONALES CONSISTENTES EN COADYU...,servicio profesional consistente coadyuvar man...
3,3000,3300,3390,SERVICIOS PROFESIONALES CONSISTENTES EN COADYU...,servicio profesional consistente coadyuvar log...
4,3000,3300,3390,SERVICIOS PROFESIONALES CONSISTENTES EN COADYU...,servicio profesional consistente coadyuvar log...


In [5]:
#Vemos aquellos registros que al preprocesar la descripción se vuelve nulo
df[df['clean_text'].isnull()]

,id_capitulo,id_concepto,id_partida_generica,descripcion,clean_text
1051,2000,2500,2540,13-0081/2022 REQ 63 CEMENAV081/2022 MATLES.AC...,NaN
1053,3000,3500,3550,13-0227/2022 REQ 322 AERO018/2022 MANTTO.CONSE...,NaN
1057,3000,3500,3580,13-0187/2022 REQ 183 CTL00011OR SERV.DE LA V....,NaN
1062,3000,3500,3550,13-0223/2022 REQ 222 AERO013/2022 MANTTO.CONSE...,NaN
1071,3000,3100,3170,13-0126/2022 REQ 155 INFO023/2022 SERV.CONDUC....,NaN
...,...,...,...,...,...
622396,2000,2500,2530,AA-050GYR018-E255-2021,NaN
623892,3000,3500,3510,LO-050GYR978-E2-2021,NaN
624710,5000,5400,5410,PC-007000999-E41-2022,NaN
624718,2000,2400,2460,PC-007000999-E53-2022,NaN


In [6]:
#Vemos la cantidad de registros iniciales
df.groupby(['id_capitulo']).count()

,id_concepto,id_partida_generica,descripcion,clean_text
id_capitulo,,,,
2000,271604,271604,271604,265502
3000,269757,269757,269757,267618
5000,17605,17605,17605,17068
6000,67202,67202,67202,66879


In [7]:
#Vemos aquellos registros que al preprocesar la descripción se vuelve nulo
df[df['clean_text'].isnull()].groupby(['id_capitulo']).count()

,id_concepto,id_partida_generica,descripcion,clean_text
id_capitulo,,,,
2000,6102,6102,6102,0
3000,2139,2139,2139,0
5000,537,537,537,0
6000,323,323,323,0


In [8]:
#Filtramos los nulos
df = df[df['clean_text'].notnull()]

In [9]:
#Vemos si han sido eliminados
df[df['clean_text'].isnull()]

,id_capitulo,id_concepto,id_partida_generica,descripcion,clean_text


In [10]:
#Guardamos los registros
df.to_csv('export_expedientes_cleaned.csv', sep=',', encoding='UTF8', index=False)